In [110]:
import arrow as A
import scipy
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import itertools
import torch
import torchvision
import xarray as xr
import glob

torch.cuda.empty_cache()
print(f'( {torch.cuda.device_count()} ) {torch.cuda.get_device_name(0)} available')

def write(x,y):
    with open(x,'a') as f:
        f.write(y)
        f.write('\n')
        f.close()
    return _


# def bandpass(x):
#     mu = np.nanmean(x)
#     sig = np.nanstd(x)
#     return mu + (3*sig)

# def xarr2torch(x):
#         return torch.from_numpy(np.asarray(x)).unsqueeze(0).unsqueeze(0).float()
    
# def mse(x,y):
#     e = y-x
#     se = e**2
    
#     valcount = torch.where(torch.isnan(se)==False,1,0)
#     valcount = torch.sum(valcount)

#     sse = np.nansum(se)
#     mse = np.nanmean(sse)
#     return mse, valcount

# def rmse(x,y):
#     e = y-x
#     se = e**2
    
#     valcount = torch.where(torch.isnan(se)==False,1,0)
#     valcount = torch.sum(valcount)

#     sse = np.nansum(se)
#     mse = np.nanmean(sse)
#     rmse = np.sqrt(mse)
#     return rmse, valcount

def fill_NOwhiten(x):
    a1 = torch.nanmean(x)
    # b1 = np.nanstd(x)
    y = torch.where(torch.isfinite(x)==False,a1,x)
    # y = (y-a1)/b1
    return y,a1

class Lnet(torch.nn.Module):
    def __init__(self,XXXX,XX):
        super().__init__()
        # in layer
        flow_channels = 8
        self.conv1 = torch.nn.Conv2d(in_channels=2, out_channels=flow_channels, kernel_size=3, padding=1, bias=False)
        self.relu1 = torch.nn.ReLU(inplace=True)
        # hidden layers
        hidden_layers = []
        for i in range(2):
            hidden_layers.append(torch.nn.Conv2d(in_channels=flow_channels, out_channels=flow_channels, kernel_size=3, padding=1, bias=False))
            hidden_layers.append(torch.nn.BatchNorm2d(flow_channels))
            hidden_layers.append(torch.nn.ReLU(inplace=True))
        self.mid_layer = torch.nn.Sequential(*hidden_layers)
        # out layer
        self.conv3 = torch.nn.Conv2d(in_channels=flow_channels, out_channels=1, kernel_size=3, padding=1, bias=False)
        
        # out layer
        self.linear1 = torch.nn.Linear(XXXX,100) ### this is what gets changed based on important switch
        self.linear2 = torch.nn.Linear(100,50)
        self.linear3 = torch.nn.Linear(50,20)
        self.linear4 = torch.nn.Linear(20,10)
        self.linear5 = torch.nn.Linear(10,XX)

    def forward(self, x):
        out1 = self.relu1(self.conv1(x))
        out = self.mid_layer(out1)
        o = self.conv3(out+out1)
        o = self.linear1(o.view(o.size(0),-1))
        o = self.relu1(self.linear2(o))
        o = self.relu1(self.linear3(o))
        o = self.relu1(self.linear4(o))
        o = self.linear5(o)
        return o
    
class dset_maker(torch.utils.data.Dataset):
    def __init__(self,x,y):
        self.x = torch.load(x)
        self.y = torch.load(y)
        self.x,self.mean = fill_NOwhiten(self.x)
        self.y,_ = fill_NOwhiten(self.y)
    
#         rand_pts = torch.from_numpy(np.random.randint(0,self.x.shape[0],100))
        
#         self.x = self.x[rand_pts]
#         self.y = self.y[rand_pts]
        
#         self.combo = torch.cat((self.x.unsqueeze(0),self.y.unsqueeze(0)),0)
        
#         RVF = torchvision.transforms.RandomVerticalFlip()
#         RHF = torchvision.transforms.RandomHorizontalFlip()
        
#         self.combo_transformed = RVF(RHF(self.combo))
#         self.x = self.combo_transformed[0]
#         self.y = self.combo_transformed[1]
        
    def __getitem__(self,idx):
        x = self.x[idx].to('cuda')
#         x -= torch.min(x)
#         x /= torch.max(x)
        y = self.y[idx].to('cuda')
#         y -= torch.min(y)
#         y /= torch.max(y)

        return x, y
    def __len__(self):
#         print(self.x.shape[0])
        return self.x.shape[0]

# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         torch.nn.init.normal_(m.weight.data,0.0,0.02)
#     elif classname.find('BatchNorm') != -1:
#         torch.nn.init.normal_(m.weight.data,1.0,0.02)
#         torch.nn.init.constant_(m.bias.data,0)  

        
class small_dset_creator(torch.utils.data.Dataset):
    def __init__(self,x,y):
        self.x = torch.load(x)[-1].reshape(-1,1,100,100)
        # self.x0 = torch.clone(self.x)
        self.y = torch.load(y)[-1].reshape(-1,1,100,100)
        # self.y0 = torch.clone(self.y)
        self.x,self.mean = fill_NOwhiten(self.x)
        # self.y,_ = fill_NOwhiten(self.y)
        
    def __getitem__(self,idx):
        x = self.x[idx].to('cuda')
        y = self.y[idx].to('cuda')
        x0 = self.x0[idx]
        y0 = self.y0[idx]
        mean = self.mean
        return x,y
    def __len__(self):
        return self.x.shape[0]  
    

        
# def foldit(x):
#     x0 = x.reshape(-1,1,x.shape[2]*x.shape[3]).permute(1,2,0)
#     fold = torch.nn.Fold(output_size=(land_mask.shape[2],land_mask.shape[3]),kernel_size=(patchsize,patchsize),stride=(patchsize,patchsize))
#     return fold(x0)

def train(model,epoch):
    epoch_loss = 0
    c=0
    for x,y in train_dataloader:
        c+=1

        # target = y-x
        predicted = model(x.cuda())

        loss = criterion(predicted, y.cuda())
        epoch_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    # scheduler.step()
    print(f'Epoch {epoch} Train Mean MSE: {np.around(epoch_loss/len(train_dataloader),3)}')
    
def validate(model,epoch):
    # validate.avg_psnr = 0
    # avg_psnr = 0
    model.eval()
    with torch.no_grad():
        t_loss = 0
        for xx,yy in valid_dataloader:
            # target = yy-xx
            predicted = model(xx.cuda())
            mse = criterion(predicted, yy.cuda())
            t_loss += mse.item()
            # modis = yy.cpu().numpy().squeeze().squeeze();
            # amsre = xx.cpu().numpy().squeeze();
            # adj = predicted.cpu().numpy().squeeze();
            # # psnr = 10 * np.log10(np.power(np.max(modis),2) / mean_squared_error(modis,amsre+adj))
            # validate.avg_psnr += (psnr / len(valid_dataloader))
        
    print(f"Epoch {epoch} Valid Mean MSE: {np.around(t_loss/len(valid_dataloader),3)}")
    
# def save_checkpoint(state):
#     model_out_path = "model_epoch_{}.pth".format(epoch)
#     torch.save(state, model_out_path)
#     print(f"Checkpoint saved to {model_out_path}")

( 1 ) NVIDIA GeForce GTX 1080 Ti available


In [106]:

# x = glob.glob('../data/traintest/*.pt')
# for i in x:
#     print(i)
#     j = torch.load(i)
#     print(j.shape)

X0 = '../data/traintest/COL_CLIP_traintest.pt'
Y0 = '../data/traintest/COL_STFL_traintest.pt'

# Y0.shape

torch.cuda.empty_cache()
print(f'( {torch.cuda.device_count()} ) {torch.cuda.get_device_name(0)} available')

lr = 1e-3
num_epochs=100
fr = .02 #colorbar fraction
blackRed = mpl.colors.ListedColormap(['red','black'])
resultFolder = '../data/traintest_results/'
train_size_prompt = float(input('% training size (valid and test split equally from here)'))

dset = dset_maker(X0,Y0)
train_size = int(train_size_prompt*len(dset))
valid_size = int(0.5*(len(dset) - train_size))
test_size = int(len(dset)-train_size-valid_size)

print(train_size,valid_size,test_size)

( 1 ) NVIDIA GeForce GTX 1080 Ti available


% training size (valid and test split equally from here) .8


168 21 21


In [107]:
train_dset = torch.utils.data.TensorDataset(dset.x[:train_size],dset.y[:train_size])
valid_dset = torch.utils.data.TensorDataset(dset.x[train_size: train_size + valid_size],
                                            dset.y[train_size: train_size + valid_size])
test_dset = torch.utils.data.TensorDataset(dset.x[train_size + valid_size:],
                                            dset.y[train_size + valid_size:])
print(len(train_dset),len(valid_dset),len(test_dset))



# test_dset.to(device)

168 21 21


In [102]:
torch.cuda.is_available()

True

In [111]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_dataloader = torch.utils.data.DataLoader(dataset=train_dset, 
                                                   batch_size=2, shuffle=False, num_workers=0)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dset, 
                                                   batch_size=1, shuffle=False, num_workers=0)
model = Lnet(dset.x.shape[2] * dset.x.shape[3],dset.y.shape[-1]).cuda()
# model.apply(weights_init)
model.cuda()
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,8], gamma=.1,verbose=True)

In [112]:
t0 = A.now('US/Eastern')

for epoch in range(1, num_epochs + 1):
    if epoch == 1:
        validate(model,epoch)
    train(model,epoch)
    if epoch % 10 == 0:
        validate(model,epoch)
t1 = A.now('US/Eastern')

# ds = small_dset_creator(amsre_lox[i],modis_lox[i])
# dl = torch.utils.data.DataLoader(ds,shuffle=False)

# model.eval()
# pred = torch.empty(0,patchsize,patchsize)
# with torch.no_grad():
#     for idx,(x,y) in enumerate(dl):
#         # target = y-x
#         predicted = model(x)
#         predicted = predicted.to('cpu')[0,0]
#         predicted = torch.from_numpy(np.where(mask_patches[idx,0]>=.5, np.nan,predicted))
#         if idx == 0:
#             pred = predicted.unsqueeze(0).unsqueeze(1)
#         else:
#             pred = torch.cat((pred,predicted.unsqueeze(0).unsqueeze(1)),0)



# filter_coast = torch.from_numpy(np.where(np.abs(pred) > np.abs(bandpass(pred)),np.nan,pred))

# ##land mask is key

# argo = torch.load(argo_lox[i])[-1] #-1 selects december

# x0 = foldit(argo) #argo
# x1 = foldit(ds.x0) #amsre
# x2 = foldit(ds.y0) #modis
# x3 = foldit(pred) #pred 
# x4 = foldit(filter_coast) #filtered pred

# xs = [x0,x1,x2,x3,x4]
# nm = ['argo','amsre','modis','pred','optim']

# fig = plt.figure(figsize=(15,5))
# gs = gridspec.GridSpec(3,5)
# for j in range(5):
#     fig.add_subplot(gs[0,j])
#     plt.title(nm[j])
#     plt.imshow(xs[j][0,0],cmap='jet',vmin=10,vmax=30)
#     plt.axis('off')
#     if j == 4:
#         plt.colorbar(fraction=.02)

#     fig.add_subplot(gs[1,j])
#     plt.title(f"{nm[j]}-{nm[0]}")
#     plt.imshow(xs[j][0,0]-xs[0][0,0],cmap='turbo_r',vmin=-2,vmax=2)
#     plt.axis('off')

#     if j == 4:
#         plt.colorbar(fraction=.02)

#     fig.add_subplot(gs[2,j])
#     plt.title(f"{nm[j]}-{nm[2]}")
#     plt.imshow(xs[j][0,0]-xs[2][0,0],cmap='turbo_r',vmin=-2,vmax=2)
#     plt.axis('off')

#     if j == 4:
#         plt.colorbar(fraction=.02)
# plt.savefig(f'trapdoor/zultz/{i}.png',bbox_inches='tight')
# plt.pause(0.05)


# write(f'trapdoor/zultz/{i}res.txt',f'dif between variable and argo')
# for idk,k in enumerate(xs):
#     write(f'trapdoor/zultz/{i}res.txt',f'{nm[idk]}\t\t{rmse(k,x0)}')

# write(f'trapdoor/zultz/{i}res.txt','\n')

# write(f'trapdoor/zultz/{i}res.txt',f'dif between variable and modis')
# for idl,l in enumerate(xs):
#     write(f'trapdoor/zultz/{i}res.txt',f'{nm[idl]}\t\t{rmse(l,x2)}')

Epoch 1 Valid Mean MSE: 21033773056.0
Epoch 1 Train Mean MSE: 15862433478.095
Epoch 2 Train Mean MSE: 15862417356.19
Epoch 3 Train Mean MSE: 15862402099.81
Epoch 4 Train Mean MSE: 15862386800.762
Epoch 5 Train Mean MSE: 15862371894.857
Epoch 6 Train Mean MSE: 15862356760.381
Epoch 7 Train Mean MSE: 15862341641.143
Epoch 8 Train Mean MSE: 15862326387.81
Epoch 9 Train Mean MSE: 15862311411.81
Epoch 10 Train Mean MSE: 15862296423.619
Epoch 10 Valid Mean MSE: 21033593429.333
Epoch 11 Train Mean MSE: 15862281325.714
Epoch 12 Train Mean MSE: 15862266227.81
Epoch 13 Train Mean MSE: 15862251157.333
Epoch 14 Train Mean MSE: 15862235931.429
Epoch 15 Train Mean MSE: 15862221068.19
Epoch 16 Train Mean MSE: 15862205985.524
Epoch 17 Train Mean MSE: 15862191018.667
Epoch 18 Train Mean MSE: 15862175948.19
Epoch 19 Train Mean MSE: 15862160612.571
Epoch 20 Train Mean MSE: 15862145779.81
Epoch 20 Valid Mean MSE: 21033425371.429
Epoch 21 Train Mean MSE: 15862130730.667
Epoch 22 Train Mean MSE: 15862115654

KeyboardInterrupt: 

In [18]:
import importlib

importlib.reload(f2f)

( 1 ) NVIDIA GeForce GTX 1080 Ti available


<module 'f2f' from '/work/albertl_uri_edu/f2f_holistic/scripts/f2f.py'>